# Sprint 16: Advanced machine learning

#### Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler

I will analyze dataset about Catalonia waste management
 - https://analisi.transparenciacatalunya.cat/Medi-Ambient/Estad-stiques-de-residus-municipals/69zu-w48s

In [6]:
df = pd.read_csv("waste_catalonia.csv")
df.head(3)

,Any,Codi municipi,Municipi,Comarca,Població,Autocompostatge,Matèria orgànica,Poda i jardineria,Paper i cartró,Vidre,...,Kg/hab/any recollida selectiva,Resta a Dipòsit,Resta a Incineració,Resta a Tractament Mecànic Biològic,Resta (sense desglossar),Suma Fracció Resta,F.R. / R.M. %,Generació Residus Municipal Totals,Kg / hab / dia,Kg / hab / any
0,2000,430017.0,Aiguamúrcia,Alt Camp,614,NaN,0.0,0.0,12.51,37.0,...,86.92,NaN,NaN,NaN,"417,61",417.61,88.67,470.98,2.10,767.07
1,2000,430056.0,Alcover,Alt Camp,3523,NaN,0.0,0.0,79.44,81.0,...,77.15,NaN,NaN,NaN,"1340,52",1340.52,83.14,1612.33,1.25,457.66
2,2000,430108.0,Alió,Alt Camp,339,NaN,0.0,0.0,4.63,7.0,...,38.42,NaN,NaN,NaN,"102,04",102.04,88.68,115.07,0.93,339.43


In [7]:
df.describe()

,Any,Codi municipi,Població,Matèria orgànica,Poda i jardineria,Paper i cartró,Vidre,Envasos lleugers,Residus voluminosos + fusta,Total Recollida Selectiva,R.S. / R.M. % total,Kg/hab/any recollida selectiva,Suma Fracció Resta,F.R. / R.M. %,Generació Residus Municipal Totals,Kg / hab / dia,Kg / hab / any
count,18947.000000,18927.000000,1.894700e+04,18947.000000,18947.000000,18947.000000,18947.000000,18947.000000,18947.000000,18947.000000,18947.000000,18947.000000,18947.000000,18947.000000,18947.000000,18947.000000,18947.000000
mean,2009.502560,211801.607281,7.588378e+03,308.761496,83.373566,349.154306,171.667322,112.512713,178.883516,1387.909978,30.232050,161.780772,2760.674036,69.657114,4148.584000,1.528665,557.967118
std,5.766987,125148.354284,5.503395e+04,3089.748431,431.780626,4682.506542,1516.320243,796.153346,1245.034188,10363.419969,18.475454,115.437389,19529.017242,18.593340,28232.508828,0.656798,239.741429
min,2000.000000,80018.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-0.140000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2005.000000,82397.000000,3.110000e+02,0.000000,0.000000,7.730000,8.665000,3.660000,0.150000,33.610000,15.480000,80.370000,107.565000,58.640000,161.980000,1.150000,418.555000
50%,2010.000000,171753.000000,9.060000e+02,9.360000,0.000000,22.450000,22.760000,12.060000,7.440000,112.050000,27.500000,139.420000,308.670000,72.460000,456.640000,1.370000,498.710000
75%,2015.000000,252075.000000,3.487500e+03,120.035000,17.000000,97.780000,80.085000,50.855000,56.055000,556.680000,41.290000,215.805000,1281.880000,84.470000,1950.845000,1.690000,617.855000
max,2019.000000,439076.000000,1.636762e+06,128393.620000,13457.290000,178024.000000,62326.000000,25792.810000,51485.300000,327502.890000,100.000000,2187.450000,690308.000000,100.000000,902356.180000,8.720000,3183.920000


I will use only some of the values to create machine learning model.
- Year
- Minicipal code
- Population
- Total waste generation in kg
- Waste generated each day by 1 habitant in each region in kg
- Sum of waste collected by selection in kg
- Percentatge of waste collected by selection (100% is total amount of waste)
- Sum of waste collected by no selection in kg
- Percentatge of waste collected by no selection (100% is total amount of waste)

In [9]:
df.columns

Index(['Any', 'Codi municipi', 'Municipi', 'Comarca', 'Població',
       'Autocompostatge', 'Matèria orgànica', 'Poda i jardineria',
       'Paper i cartró', 'Vidre', 'Envasos lleugers',
       'Residus voluminosos + fusta', 'RAEE', 'Ferralla', 'Olis vegetals',
       'Tèxtil', 'Runes', 'Residus Especials en petites quantitats (REPQ',
       'Piles', 'Medicaments', 'Altres recollides selectives',
       'Total Recollida Selectiva', 'R.S. / R.M. % total',
       'Kg/hab/any recollida selectiva', 'Resta a Dipòsit',
       'Resta a Incineració', 'Resta a Tractament Mecànic Biològic',
       'Resta (sense desglossar)', 'Suma Fracció Resta', 'F.R. / R.M. %',
       'Generació Residus Municipal Totals', 'Kg / hab / dia',
       'Kg / hab / any'],
      dtype='object')

In [13]:
df_waste = df[['Any','Codi municipi','Població','Generació Residus Municipal Totals',
                     'Kg / hab / dia','Total Recollida Selectiva','R.S. / R.M. % total',
                     'Suma Fracció Resta', 'F.R. / R.M. %']]              

In [17]:
df_waste.head(2)

,Any,Codi municipi,Població,Generació Residus Municipal Totals,Kg / hab / dia,Total Recollida Selectiva,R.S. / R.M. % total,Suma Fracció Resta,F.R. / R.M. %
0,2000,430017.0,614,470.98,2.10,53.37,11.33,417.61,88.67
1,2000,430056.0,3523,1612.33,1.25,271.81,16.86,1340.52,83.14


In [18]:
df_final = df_waste.rename({'Any': 'Year', 'Codi municipi': 'Municipal_Code',
                           'Població': 'Population', 'Generació Residus Municipal Totals': 'Total_waste',
                           'Kg / hab / dia': 'Kg_person_day', 'Total Recollida Selectiva': 'Selecting',
                           'R.S. / R.M. % total': 'Perc_selecting', 'Suma Fracció Resta': 'No_selecting',
                           'F.R. / R.M. %': 'Perc_no_selecting',}, axis='columns')
df_final.head(2)

,Year,Municipal_Code,Population,Total_waste,Kg_person_day,Selecting,Perc_selecting,No_selecting,Perc_no_selecting
0,2000,430017.0,614,470.98,2.10,53.37,11.33,417.61,88.67
1,2000,430056.0,3523,1612.33,1.25,271.81,16.86,1340.52,83.14


## Level 1

### Exercise 1

#### Aplication of Pipeline and GridSearch with Random Forest

### Exercise 2

#### Frequency of words in english text

## Level 2

### Exercise 3

#### Stopwords, stemming

## Level 3

### Exercise 4

#### Sentiment analysis

#### Notes
* Pipelines
    - https://towardsdatascience.com/a-simple-example-of-pipeline-in-machine-learning-with-scikit-learn-e726ffbb6976
    - https://www.analyticsvidhya.com/blog/2020/01/build-your-first-machine-learning-pipeline-using-scikit-learn/
* Parameters tuning
    - https://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/
* Text analytics
    - https://www.datacamp.com/community/tutorials/text-analytics-beginners-nltk
    - https://www.nltk.org